In [1]:
import re
import sys
import random
import os
import subprocess as sp
from multiprocessing.pool import Pool
import shutil
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
import functools

import find_trap as findtrap
import run_output_bonds_func as robf
import convexhull as cvh
import vista_func as vf
import get_coordinate as gc
import config as cfg
importlib.reload(findtrap)
importlib.reload(robf)
importlib.reload(cvh)
importlib.reload(vf)
importlib.reload(cfg)

<module 'config' from '/Users/takepy/takeoxdna/kakenhievolvedna/oxdna_run/config.py'>

In [2]:
import main

In [3]:
def make_domain_list(linelist, length_tup):
    index_domains = []
    
    count_start = 0
    
    for block in linelist:
        print("block : ", block)

        #[('a*', 0), ('b', 1)]
        """
        block0 = list(block[0])
        #block1 = list(block[1])だったが、長さ3以上の構造に対応できなかった。
        block1 = list(block[-1])#ブロックの最後尾を取ることにした

        index_domain1 = [block0[0], block0[1], count_start, count_end]       
        count_start +=length_tup[0]
        count_end += length_tup[0]
        index_domain2 = [block1[0], block1[1], count_start, count_end] 
        count_start +=length_tup[1]
        count_end += length_tup[1]
        print("index_domain : ", index_domain1, index_domain2)
        index_domain.append(index_domain1)
        index_domain.append(index_domain2)
        """
        
        for b in block:
            count_end = count_start + length_tup[b[0][0]]
            index_domains.append([b[0],b[1],count_start,count_end-1])
            count_start = count_end
        
        #index_domain.append( index_domain1 + index_domain2 + [block])
        print(index_domains)
    return index_domains

In [4]:
def make_domain_df(lst):
    df2 = pd.DataFrame(lst, columns =["domain", "num", "start_index", "end_index"])
    return df2

In [5]:
def drop_not_connected(df):
    newdf = df[df['num'] >= 0]
    return newdf

In [6]:
def make_groups(df):
    pairs = df.groupby(["num"])
    return pairs


In [7]:
def count_domains(df):
    df_pairs = pd.DataFrame(df["num"].value_counts())
    return len(df_pairs)

In [8]:
def make_external(df, target, output_dir):
            
    #output_path = "traps/external_{}.conf".format(target)
    output_path = os.path.join(output_dir, "external_{}.conf".format(target))
    external_file = open(output_path, "w")
    domains_num = count_domains(df)
    pairs = make_groups(df)
    print(pairs.groups)
    
    for x in range (0, domains_num):        
        print("roop", x)
        group = pairs.get_group(x)
        print("group:\n", group, "\n")
        start_index_min = group["start_index"].min()
        end_index_max =  group["end_index"].max()
        print("index : ", start_index_min, " " , end_index_max)
        lines = ["{\n","type = mutual_trap\n","particle = {}\n".format(start_index_min),"ref_particle = {}\n".format(end_index_max),"stiff = 1.\n","r0 = 1.2\n","}\n""{\n", "type = mutual_trap\n","particle = {}\n".format(end_index_max),"ref_particle = {}\n".format(start_index_min),"stiff = 1.\n","r0 = 1.2\n","}\n"]
        external_file.writelines(lines)
    external_file.close()

In [9]:
def make_trap(linelist, length_tup, target, output_dir):
    #print(linelist, "\n")
    domain_list = make_domain_list(linelist, length_tup)
    print("domain_list : ", domain_list, "\n")
    domain_df = make_domain_df(domain_list)
    print("domain_df : \n", domain_df)
    connected_df = drop_not_connected(domain_df)
    print("connected_df : \n", connected_df)
    make_external(connected_df, target, output_dir)
    return domain_list


In [10]:
string = "e3001 = a* b( a*( ) + ) a* a b*( + a* ) a* a"
#string = "e24 = a( b( + a* ) + ) b*( + a )"
#replaced_linelist = main.replace_parentheses(string)
#display(replaced_linelist)

In [13]:
length_a = 6
length_b = 7
length_tup = {"a":length_a, "b":length_b}
str_a = main.get_random_DNA(length_a)
str_b = main.get_random_DNA(length_b)
str_a_star = main.get_comp_DNA(str_a)
str_b_star = main.get_comp_DNA(str_b)
output_ATGC_folder = "."

In [14]:
filename, replaced_linelist = main.write_file(string, str_a, str_b, str_a_star, str_b_star, output_ATGC_folder)
print(filename, "\n", replaced_linelist)
domain_list = make_domain_list(replaced_linelist, length_tup)
display(domain_list)

replace_parentheses() : start

index, character :  0  ,   
accumulate :  []
current_strand :  [] 

index, character :  1  ,  a
accumulate :  ['a']
current_strand :  [] 

index, character :  2  ,  *
accumulate :  ['a', '*']
current_strand :  [] 

index, character :  3  ,   
accumulate :  []
current_strand :  [('a*', -1)] 

index, character :  4  ,  b
accumulate :  ['b']
current_strand :  [('a*', -1)] 

index, character :  5  ,  (
accumulate :  []
current_strand :  [('a*', -1), ('b', 0)] 

index, character :  6  ,   
accumulate :  []
current_strand :  [('a*', -1), ('b', 0)] 

index, character :  7  ,  a
accumulate :  ['a']
current_strand :  [('a*', -1), ('b', 0)] 

index, character :  8  ,  *
accumulate :  ['a', '*']
current_strand :  [('a*', -1), ('b', 0)] 

index, character :  9  ,  (
accumulate :  []
current_strand :  [('a*', -1), ('b', 0), ('a*', 1)] 

index, character :  10  ,   
accumulate :  []
current_strand :  [('a*', -1), ('b', 0), ('a*', 1)] 

index, character :  11  ,  )
accu

[['a*', -1, 0, 5],
 ['b', 0, 6, 12],
 ['a*', 1, 13, 18],
 ['a', 1, 19, 24],
 ['b*', 0, 25, 31],
 ['a*', -1, 32, 37],
 ['a', -1, 38, 43],
 ['b*', 2, 44, 50],
 ['a*', -1, 51, 56],
 ['b', 2, 57, 63],
 ['a*', -1, 64, 69]]

 [[('a*', 0), ('b', 1), ('a*', 2), ('a', 2)], [('b*', 1), ('a*', -1), ('a', 0), ('b*', 3)], [('a*', -1), ('b', 3), ('a*', 4), ('a', 4)]]
は、(domain, num)についてnumが"(", ")"の関係に対応する。同じnumのドメイン同士が結合している

In [ ]:
domain_list